The following code blocks aim at mergin the pollen and weather data for all the municipalities and put them in a daily format

In [8]:
# Count the number of rows in the CSV file
import pandas as pd
import os

zone_mapping_file = '/Users/davidegiordani/Desktop/BGTFinalCrack/TrentinoAltoAdigeZoneMapping/Final/Merged_Valleys_and_Municipalities.csv'
pollen_files = [f for f in os.listdir('/Users/davidegiordani/Desktop/BGTFinalCrack/WeatherTrentinoAltoAdige2024') if f.endswith('.csv')]
zone_mapping_df = pd.read_csv(zone_mapping_file)
num_rows = len(zone_mapping_df)
print(f"Number of rows in '{zone_mapping_file}': {num_rows}")

# Count the number of files in the PollenTrentinoAltoAdige2024 folder
num_pollen_files = len(pollen_files)
print(f"Number of files in 'PollenTrentinoAltoAdige2024': {num_pollen_files}")

Number of rows in '/Users/davidegiordani/Desktop/BGTFinalCrack/TrentinoAltoAdigeZoneMapping/Final/Merged_Valleys_and_Municipalities.csv': 598
Number of files in 'PollenTrentinoAltoAdige2024': 464


In [23]:
import os
import pandas as pd

# Directories
weather_folder = '/Users/davidegiordani/Desktop/BGTFinalCrack/WeatherTrentinoAltoAdige2024'
pollen_folder = '/Users/davidegiordani/Desktop/BGTFinalCrack/PollenTrentinoAltoAdige2024'
output_folder = '/Users/davidegiordani/Desktop/BGTFinalCrack/MergedDataTrentinoAltoAdige2024'
os.makedirs(output_folder, exist_ok=True)

# Get list of files
weather_files = [f for f in os.listdir(weather_folder) if f.endswith('_weather.csv')]
pollen_files = [f for f in os.listdir(pollen_folder) if f.endswith('_pollen.csv')]

# Extract base names without extensions
weather_basenames = {f.replace('_weather.csv', ''): f for f in weather_files}
pollen_basenames = {f.replace('_pollen.csv', ''): f for f in pollen_files}

# Find common municipalities
common_municipalities = set(weather_basenames.keys()) & set(pollen_basenames.keys())

# Function to merge files for a municipality
def merge_files(municipality, weather_file, pollen_file, output_file):
    weather_path = os.path.join(weather_folder, weather_file)
    pollen_path = os.path.join(pollen_folder, pollen_file)

    # Load datasets
    weather_data = pd.read_csv(weather_path)
    pollen_data = pd.read_csv(pollen_path)

    # Parse time column
    weather_data['time'] = pd.to_datetime(weather_data['time'], errors='coerce')
    pollen_data['time'] = pd.to_datetime(pollen_data['time'], errors='coerce')

    # Merge datasets on time
    merged_data = pd.merge(weather_data, pollen_data, on='time', how='inner')

    # Save to output file
    merged_data.to_csv(output_file, index=False)

# Merge files for each municipality
for municipality in common_municipalities:
    weather_file = weather_basenames[municipality]
    pollen_file = pollen_basenames[municipality]
    output_file = os.path.join(output_folder, f"{municipality}_data.csv")

    merge_files(municipality, weather_file, pollen_file, output_file)

print("All files processed and merged successfully.")


All files processed and merged successfully.


In [28]:
import os
import pandas as pd
import numpy as np

# Directories
merged_data_folder = '/Users/davidegiordani/Desktop/BGTFinalCrack/MergedDataTrentinoAltoAdige2024'
daily_data_folder = '/Users/davidegiordani/Desktop/BGTFinalCrack/MergedDataTrentinoAltoAdige2024/Daily'
os.makedirs(daily_data_folder, exist_ok=True)

# Function to compute daily averages for a municipality
def compute_daily_averages(file_path, output_path):
    # Load data
    data = pd.read_csv(file_path)

    # Ensure 'time' is parsed correctly
    data['time'] = pd.to_datetime(data['time'], errors='coerce')

    # Add a 'date' column for grouping
    data['date'] = data['time'].dt.date

    # Initialize list for daily data
    daily_records = []

    # Group by date
    for date, group in data.groupby('date'):
        daily_record = {'date': date}

        for column in group.select_dtypes(include=['float64', 'int64']).columns:
            non_nan_count = group[column].notna().sum()
            total_count = len(group[column])

            if non_nan_count >= total_count / 2:
                daily_record[column] = group[column].mean()
            else:
                daily_record[column] = np.nan

        daily_records.append(daily_record)

    # Create DataFrame for daily averages
    daily_data = pd.DataFrame(daily_records)

    # Save the daily-averaged data
    daily_data.to_csv(output_path, index=False)

# Process all municipality files
municipality_files = [f for f in os.listdir(merged_data_folder) if f.endswith('_data.csv')]

for municipality_file in municipality_files:
    file_path = os.path.join(merged_data_folder, municipality_file)
    output_path = os.path.join(daily_data_folder, municipality_file)

    compute_daily_averages(file_path, output_path)

print("Daily averages computed and saved for all municipalities.")


Daily averages computed and saved for all municipalities.


In [32]:
import os
import pandas as pd

# Directories
daily_data_folder = '/Users/davidegiordani/Desktop/BGTFinalCrack/MergedDataTrentinoAltoAdige2024/Daily'
valleys_data_folder = '/Users/davidegiordani/Desktop/BGTFinalCrack/MergedDataTrentinoAltoAdige2024/Valleys'
os.makedirs(valleys_data_folder, exist_ok=True)

# Load valley-municipality mapping
mapping_file_path = '/Users/davidegiordani/Desktop/BGTFinalCrack/TrentinoAltoAdigeZoneMapping/Final/Merged_Valleys_and_Municipalities.csv'
mapping_data = pd.read_csv(mapping_file_path)

# Create a dictionary mapping municipalities to valleys
municipality_to_valley = mapping_data.set_index('normalized_municipality')['valley'].to_dict()

# Process all municipality files
municipality_files = [f for f in os.listdir(daily_data_folder) if f.endswith('_data.csv')]

for municipality_file in municipality_files:
    municipality_name = municipality_file.replace('_data.csv', '')
    valley_name = municipality_to_valley.get(municipality_name)

    if valley_name:
        # Create valley folder if it doesn't exist
        valley_folder = os.path.join(valleys_data_folder, valley_name)
        os.makedirs(valley_folder, exist_ok=True)

        # Move municipality file to the valley folder
        source_path = os.path.join(daily_data_folder, municipality_file)
        destination_path = os.path.join(valley_folder, municipality_file)
        os.rename(source_path, destination_path)

print("Municipality files organized into valley folders.")

Municipality files organized into valley folders.


In [42]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle

# Directories
valleys_data_folder = '/Users/davidegiordani/Desktop/BGTFinalCrack/MergedDataTrentinoAltoAdige2024/Valleys'
valleys_models_folder = '/Users/davidegiordani/Desktop/BGTFinalCrack/MergedDataTrentinoAltoAdige2024/ValleysModels1'
os.makedirs(valleys_models_folder, exist_ok=True)

# Pollen columns to predict
pollen_columns = [
    'Alder pollen (grains/m³)', 'Birch pollen (grains/m³)', 
    'Grass pollen (grains/m³)', 'Mugwort pollen (grains/m³)', 
    'Olive pollen (grains/m³)', 'Ragweed pollen (grains/m³)'
]

# Function to preprocess and train a unified model
def train_valley_model(valley_folder, output_folder, valley_name):
    # Merge all municipality data
    merged_data = pd.DataFrame()
    municipality_files = [f for f in os.listdir(valley_folder) if f.endswith('_data.csv')]

    for municipality_file in municipality_files:
        file_path = os.path.join(valley_folder, municipality_file)
        data = pd.read_csv(file_path)
        data['municipality'] = municipality_file.replace('_data.csv', '')
        merged_data = pd.concat([merged_data, data], ignore_index=True)

    # Ensure 'date' column is in datetime format
    merged_data['date'] = pd.to_datetime(merged_data['date'], errors='coerce')

    # Initialize results
    results = {}
    models = {}

    # Iterate over each pollen type
    for pollen_type in pollen_columns:
        if pollen_type not in merged_data.columns:
            print(f"Warning: {pollen_type} is missing in the dataset. Skipping...")
            continue

        # Fill missing pollen values
        merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')

        # Create lagged features
        merged_data[f'{pollen_type}_lag_1'] = merged_data[pollen_type].shift(1)
        merged_data[f'{pollen_type}_lag_2'] = merged_data[pollen_type].shift(2)
        merged_data[f'{pollen_type}_lag_3'] = merged_data[pollen_type].shift(3)

        # Drop rows with NaN values due to lagging
        data_cleaned = merged_data.dropna()

        # Define features and target
        pollen_lagged_features = [f'{pollen_type}_lag_1', f'{pollen_type}_lag_2', f'{pollen_type}_lag_3']
        weather_features = [col for col in merged_data.columns if col.startswith('temperature') or col.startswith('precipitation')]
        X = data_cleaned[pollen_lagged_features + weather_features]
        y = data_cleaned[pollen_type]

        # Split data into train-test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train Random Forest model
        rf = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=None)
        rf.fit(X_train, y_train)

        # Evaluate the model
        y_pred = rf.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Save results and model
        results[pollen_type] = {'RMSE': rmse, 'MAE': mae, 'R2': r2}
        models[pollen_type] = rf

        print(f"{pollen_type} - RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.2f}")

    # Save results to CSV
    results_path = os.path.join(output_folder, f'{valley_name}_results.csv')
    pd.DataFrame(results).T.to_csv(results_path)

    # Save models to a pickle file with valley name
    models_path = os.path.join(output_folder, f'{valley_name}_models.pkl')
    with open(models_path, 'wb') as f:
        pickle.dump(models, f)

    # Copy models to a central folder
    central_model_path = os.path.join(valleys_models_folder, f'{valley_name}_models.pkl')
    with open(central_model_path, 'wb') as f:
        pickle.dump(models, f)

    print(f"Models and results saved for valley: {valley_name} and copied to ValleysModels")

# Process all valleys
for valley_name in os.listdir(valleys_data_folder):
    valley_folder = os.path.join(valleys_data_folder, valley_name)
    if os.path.isdir(valley_folder):
        print(f"Processing valley: {valley_name}")
        output_model_folder = os.path.join(valley_folder, 'Models')
        os.makedirs(output_model_folder, exist_ok=True)
        train_valley_model(valley_folder, output_model_folder, valley_name)


Processing valley: Val Venosta


/var/folders/q2/9lh55t_s1k56_qhpfd2m00f80000gn/T/ipykernel_921/3166904741.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')


Alder pollen (grains/m³) - RMSE: 0.25, MAE: 0.05, R2: 0.99


/var/folders/q2/9lh55t_s1k56_qhpfd2m00f80000gn/T/ipykernel_921/3166904741.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')


Birch pollen (grains/m³) - RMSE: 2.68, MAE: 0.93, R2: 0.89


...

Grass pollen (grains/m³) - RMSE: 2.11, MAE: 0.73, R2: 0.94
Mugwort pollen (grains/m³) - RMSE: 0.00, MAE: 0.00, R2: 1.00


/var/folders/q2/9lh55t_s1k56_qhpfd2m00f80000gn/T/ipykernel_921/3166904741.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')
/var/folders/q2/9lh55t_s1k56_qhpfd2m00f80000gn/T/ipykernel_921/3166904741.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')


Olive pollen (grains/m³) - RMSE: 0.13, MAE: 0.02, R2: 0.88


/var/folders/q2/9lh55t_s1k56_qhpfd2m00f80000gn/T/ipykernel_921/3166904741.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')


Ragweed pollen (grains/m³) - RMSE: 0.14, MAE: 0.02, R2: 0.87
Models and results saved for valley: Valle di Rendena and copied to ValleysModels
